# Week 3 — Workflow Orchestration with Prefect
**Goal:** Orchestrate a tiny ETL with retries, logging, and parameters.

## 0) Setup

In [1]:
# !pip -q install prefect pandas
import pandas as pd, io, os
try:
    from prefect import flow, task, get_run_logger
    from prefect.task_runners import ConcurrentTaskRunner
except Exception:
    print("If in Colab, run: !pip install prefect")
    raise

## 1) Tasks

In [2]:
@task(retries=2, retry_delay_seconds=2)
def extract() -> pd.DataFrame:
    import random
    logger = get_run_logger()
    logger.info("Extracting...")
    if random.random() < 0.2:
        logger.warning("Transient failure; retrying...")
        raise RuntimeError("network hiccup")
    csv_text = '''order_id,customer,country,amount,ts
1001,Alice,US,120.50,2024-06-01
1002,Bob,UK,85.00,2024-06-02
1003,Chandra,IN,NaN,2024-06-02
1004,Diego,US,43.20,2024-06-03
1005,Eva,DE,69.99,2024-06-03
'''
    return pd.read_csv(io.StringIO(csv_text))

@task
def transform(df: pd.DataFrame) -> pd.DataFrame:
    logger = get_run_logger()
    logger.info("Transforming...")
    out = df.copy()
    out['amount'] = pd.to_numeric(out['amount'], errors='coerce').fillna(out['amount'].median())
    return out

@task
def load(df: pd.DataFrame, dest_path: str = "warehouse/orders_clean.csv") -> str:
    logger = get_run_logger()
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    df.to_csv(dest_path, index=False)
    logger.info(f"Wrote {len(df)} rows to {dest_path}")
    return dest_path

## 2) Flow

In [3]:
@flow(name="mini-etl", task_runner=ConcurrentTaskRunner())
def etl_flow(dest_path: str = "warehouse/orders_clean.csv") -> str:
    df = extract()
    tidy = transform(df)
    return load(tidy, dest_path)

result = etl_flow()
result

12:07:09.225 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8217
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

12:07:13.975 | INFO    | Flow run 'rich-scorpion' - Beginning flow run 'rich-scorpion' for flow 'mini-etl'

12:07:14.009 | INFO    | Task run 'extract-e1b' - Extracting...

12:07:14.010 | WARNING | Task run 'extract-e1b' - Transient failure; retrying...

12:07:14.013 | INFO    | Task run 'extract-e1b' - Task run failed with exception: RuntimeError('network hiccup') - Retry 1/2 will start 2 second(s) from now

...

12:07:16.061 | INFO    | Task run 'transform-7fc' - Finished in state Completed()

12:07:16.086 | INFO    | Task run 'load-8c5' - Wrote 5 rows to warehouse/orders_clean.csv

12:07:16.088 | INFO    | Task run 'load-8c5' - Finished in state Completed()

12:07:16.096 | INFO    | Flow run 'rich-scorpion' - Finished in state Completed()

'warehouse/orders_clean.csv'